<a href="https://colab.research.google.com/github/dk-wei/python-tricks/blob/main/Numba_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考链接：https://www.youtube.com/watch?v=x58W9A2lnQc

To better ultilize `numba`, we should always:
1. use `njit` decorator
2. prelocate input as much as we can

In [113]:
from numba import jit, njit
import random

# regular python baseline

In [125]:
# @jit(nopython=True)
def function(nsamples):
  res = 0
  for i in range(nsamples):
    x = random.random()
    y = random.random()
    if (x**2 + y**2) < 1.0:
      res +=1
  return 4*res/nsamples

In [126]:
%%time
function(100000000)

CPU times: user 44.1 s, sys: 72.5 ms, total: 44.2 s
Wall time: 44.2 s


3.14156856

# Numba `njit`

In [127]:
#@jit(nopython=True)
@njit
def function(nsamples):
  res = 0
  for i in range(nsamples):
    x = random.random()
    y = random.random()
    if (x**2 + y**2) < 1.0:
      res +=1
  return 4*res/nsamples

In [129]:
%%time
function(100000000)

CPU times: user 1.25 s, sys: 5.02 ms, total: 1.26 s
Wall time: 1.26 s


3.1417056

# Numba prelocation

In [155]:
#@jit(nopython=True)
@njit(nogil = True)
def function(nsamples):
  res = 0
  input = np.array(list(range(5)))   # simply let numba know the size of input to do prelocating
  for i in input:
    x = random.random()
    y = random.random()
    if (x**2 + y**2) < 1.0:
      res +=1
  return 4*res/nsamples

In [157]:
%%time
function(100000000)

CPU times: user 983 µs, sys: 2 µs, total: 985 µs
Wall time: 1.08 ms


1.2e-07

不支持哪些 features?
说了那么多，哪些能用，哪些不能用？当前版本（本文撰写时为 0.45）的 Numba 不支持以下 Python 功能：

- Exception handling (try .. except, try .. finally)
- Context management (the with statement)
- Dict/set/generator comprehensions
- Generator delegation (yield from)

此外，还有一些限制，例如 list/tuple 的元素必须同类（静态）等。对于 NumPy，其支持的 feature 见文末的参考链接。